# Simulation data processing and plotting

### Imports

In [1]:
# Imports and defaults
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import pygwalker as pyg

pio.templates.default='plotly_white'

### Data input cells - base data

In [2]:
# Actual ucm comp data plus some other stuff
ucm23_sim_energy = 696.64 #kJ
ucm23_real_energy = 3.055 #kWh
ucm23_real_lap_energy = ucm23_real_energy * 3600/18

ucm23_autocross_time_sim = 100.72
ucm23_autocross_time_real = 96
ucm23_endurance_time_real = 2061
ucm23_skidpad_time_sim = 4.89
ucm23_skidpad_time_real = 5.25
ucm23_accel_time_sim = 4.286 # from john sim
ucm23_accel_time_real = 4.24



### Create car class

In [3]:
class Car:

    def __init__(self, name, skidpadtime,acceltime,autocrosstime,energyperlap):
        self.name = name
        self.skidpadtime = skidpadtime
        self.acceltime = acceltime
        self.autocrosstime = autocrosstime
        self.energyperlap = energyperlap

        endurance_time = (self.autocrosstime/ucm23_autocross_time_sim) * ucm23_endurance_time_real
        efficiency_kWh = (self.energyperlap/ucm23_sim_energy) * ucm23_real_energy
        skidpad_time = (self.skidpadtime/ucm23_skidpad_time_sim) * ucm23_skidpad_time_real
        accel_time = (self.acceltime/ucm23_accel_time_sim) * ucm23_accel_time_real
        autocross_time = (self.autocrosstime/ucm23_autocross_time_sim) * ucm23_autocross_time_real

        self.calc_endurance_points(endurance_time)
        self.calc_efficiency_points(endurance_time, efficiency_kWh)
        self.calc_accel_points(accel_time)
        self.calc_skidpad_points(skidpad_time)
        self.calc_autocross_points(autocross_time)
        self.points_total = self.accel_points + self.autocross_points + self.efficiency_points + self.skidpad_points + self.endurance_points
    
    def calc_endurance_points(self, time):
        t_min_end = 1961
        t_max_end = 1.45 * t_min_end
        self.endurance_points = 250 * (((t_max_end/time)-1)/((t_max_end/t_min_end)-1))
    
    def calc_efficiency_points(self, time, efficiency):
        endurance_distance = 21.36 # distance of the full course in km

        your_laps = 18 # Number of laps completed - assume finish endurance
        co2_per_kwh = 0.65 # kg per kwh
        your_c02 = efficiency * co2_per_kwh

        t_min_efficiency = 1961 # Time elapsed by the team with minimum efficiency time (incl dnfs, but must be efficiency eligible)
        laps_tmin = 18 # Number of laps completed by team with minimum endurance time ******

        laps_co2min = 18 # Number of laps completed by team with min co2 score
        co2_min = 2.59*co2_per_kwh # kg of co2 equivalent minimum from teams

        your_efficiency_factor = ((t_min_efficiency/laps_tmin)/(time/your_laps)) * ((co2_min/laps_co2min)/(your_c02/your_laps))
        min_efficiency_factor = ((t_min_efficiency/laps_tmin)/(1.45*t_min_efficiency/18)) * ((co2_min/laps_co2min)/(20.02*endurance_distance/18))
        efficiency_factor_max = ((t_min_efficiency/laps_tmin)/(2359/18)) * ((co2_min/laps_co2min)/(0.65*2.59/18)) # Max eff factor of all the teams - UNISA

        efficiency_points = 100 * (your_efficiency_factor - min_efficiency_factor)/(efficiency_factor_max - min_efficiency_factor)
        if efficiency_points >= 100:
            efficiency_points = 100

        self.efficiency_points = efficiency_points

    def calc_accel_points(self, time):
        t_min_acc = 3.69 # fastest accel time of all teams
        t_max_acc = 1.5 * t_min_acc
        self.accel_points = 95.5 * (((t_max_acc/time)-1)/((t_max_acc/t_min_acc)-1)) + 4.5
        if self.accel_points > 100:
            self.accel_points = 100;
    
    def calc_skidpad_points(self,time):
        t_min_skd = 5.12 # fastest skidpad time
        t_max_skd = t_min_skd * 1.25
        self.skidpad_points = 71.5 * ((t_max_skd/time)**2 - 1)/((t_max_skd/t_min_skd)**2 - 1) + 3.5
    
    def calc_autocross_points(self,time):
        t_min_aut = 90 #fastest autocross time
        t_max_aut = 1.45 * t_min_aut
        self.autocross_points = 118.5 * (((t_max_aut/time)-1)/((t_max_aut/t_min_aut)-1)) + 6.5

# Create list to store car objects in
cars_list = []

## Enter Cars by Time and Energy Results
### UCM24 Aero

In [4]:
cars_list.append(Car('UCM24 Aero', 4.725, 4.034, 97.93, 765.80))

### M023

In [5]:
cars_list.append(Car('M023', 4.89, 3.69, 97.95, 1201.1))

### UCM23 No Aero

In [6]:
cars_list.append(Car('UCM23 No Aero', 4.935, 4.3, 101.66, 660.68))

### UCM23 Standard

In [7]:
cars_list.append(Car('UCM23 Standard', 4.885, 4.286, 100.72, 696.64))

cars_list.append(Car('UCM24 No Aero', 4.885, 4.16, 100.56, 619.70))


### UCM24 No Aero ^

## Calculate points for each car and plot

In [9]:
# Assuming ucm23_* variables are defined somewhere in the global scope
# cars is a list of Car objects
# cars_list = [...]  # List of Car objects

# Create an empty list to store the data
data = []

# Loop through each car and extract the necessary information
for car in cars_list:
    data.append({
        'CarConfig': car.name,
        'Skidpad': car.skidpad_points,
        'Accel': car.accel_points,
        'Autocross': car.autocross_points,
        'Endurance': car.endurance_points,
        'Efficiency': car.efficiency_points
    })

data.append({
        'CarConfig': 'Points split',
        'Skidpad': 75,
        'Accel': 100,
        'Autocross': 125,
        'Endurance': 275,
        'Efficiency': 100
    })

# Create a pandas DataFrame from the list of dictionaries
cars_df = pd.DataFrame(data)

# Display the DataFrame
print(cars_df)
print()
for car in cars_list:
    print(f'{car.name}| Skidpad: {car.skidpadtime}  Autocross: {car.autocrosstime}  Acceleration: {car.acceltime}  Energy per lap: {car.energyperlap}')

# Plotly stacked bar chart using go.Figure to fully customize trace text
import plotly.graph_objects as go

# List of event categories
categories = ['Skidpad', 'Accel', 'Autocross', 'Endurance', 'Efficiency']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

# Create figure
fig = go.Figure()

# Add bars for each event category with individual text inside each bar
for i, category in enumerate(categories):
    fig.add_trace(go.Bar(
        x=cars_df['CarConfig'],
        y=cars_df[category],
        name=category,
        text=cars_df[category],  # Add points as text
        texttemplate='%{text:.2f}',  # Format the text to 2 decimal places
        textposition='inside',  # Position text inside the bars
        marker_color=colors[i]  # Assign different colors for each category
    ))

# Update layout for stacked bars
fig.update_layout(
    barmode='stack',
    title="Dynamic Event Points",
    xaxis_title="Car Configuration",
    yaxis_title="Points",
    legend_title="Events",
    xaxis={'categoryorder': 'total descending'},
    height=600,
    width=800
)

# Show the figure
fig.show()



        CarConfig    Skidpad       Accel   Autocross   Endurance  Efficiency
0      UCM24 Aero  78.708951   78.411864  111.333852  232.750861   90.758455
1            M023  65.285800  100.000000  111.258678  232.589900   57.735362
2   UCM23 No Aero  61.856578   62.024293   97.825456  203.827165  100.000000
3  UCM23 Standard  65.672685   62.836085  101.135417  210.914335   97.027546
4   UCM24 No Aero  65.672685   70.388091  101.704977  212.133858  100.000000
5    Points split  75.000000  100.000000  125.000000  275.000000  100.000000

UCM24 Aero| Skidpad: 4.725  Autocross: 97.93  Acceleration: 4.034  Energy per lap: 765.8
M023| Skidpad: 4.89  Autocross: 97.95  Acceleration: 3.69  Energy per lap: 1201.1
UCM23 No Aero| Skidpad: 4.935  Autocross: 101.66  Acceleration: 4.3  Energy per lap: 660.68
UCM23 Standard| Skidpad: 4.885  Autocross: 100.72  Acceleration: 4.286  Energy per lap: 696.64
UCM24 No Aero| Skidpad: 4.885  Autocross: 100.56  Acceleration: 4.16  Energy per lap: 619.7
